In [0]:
!pip3 install spotipy

In [0]:
import spotipy
import spotipy.oauth2 as oauth2
import random
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import time
import pandas as pd

# Your client id and client secret will be here.
credentials = oauth2.SpotifyClientCredentials(
        client_id='',
        client_secret='')

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [0]:
song_uri = "spotify:track:5lBQ3mnWsYIt5aCdgz1U9n"
song_response = spotify.track(song_uri)
artist_uri = song_response['artists'][0]['uri']

In [0]:
audio_features = spotify.audio_features(song_uri) 
print(audio_features)

[{'danceability': 0.456, 'energy': 0.776, 'key': 9, 'loudness': -6.999, 'mode': 0, 'speechiness': 0.0348, 'acousticness': 0.0423, 'instrumentalness': 0.00975, 'liveness': 0.148, 'valence': 0.223, 'tempo': 174.261, 'type': 'audio_features', 'id': '5lBQ3mnWsYIt5aCdgz1U9n', 'uri': 'spotify:track:5lBQ3mnWsYIt5aCdgz1U9n', 'track_href': 'https://api.spotify.com/v1/tracks/5lBQ3mnWsYIt5aCdgz1U9n', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5lBQ3mnWsYIt5aCdgz1U9n', 'duration_ms': 358600, 'time_signature': 4}]


In [0]:
start_time = time.time()

relatedList = []

related_artists = spotify.artist_related_artists(artist_uri)['artists']
relatedList.append(artist_uri)

for i in related_artists:
  uri = i['uri']
  inner_artists = spotify.artist_related_artists(uri)
  for inner_artist in inner_artists['artists']: 
    inner_uri = inner_artist['uri']
    if(inner_uri not in relatedList):
      relatedList.append(inner_uri)

relatedList = random.sample(set(relatedList), 60 if len(relatedList) >= 60 else len(relatedList))

track_list = []

track_list.append({
    'track': song_response['name'],
    'artist': song_response['artists'][0]['name'],
    'track_uri': song_response['uri']
})

for artist in relatedList:
  albums = spotify.artist_albums(artist, country='TR')
  albums = albums['items']
  iter_list = [iter_index for iter_index in range(len(albums))]
  iter_list = random.sample(set(iter_list), 3 if len(albums) >= 3 else len(iter_list))
  random_albums = []
  for iter_index in iter_list:
    random_albums.append(albums[iter_index])
  for album in random_albums:
    tracks = spotify.album_tracks(album['uri'])
    tracks = tracks['items']
    tracks_uri = []
    for track in tracks:
      track_list.append({
          'track': track['name'],
          'artist': track['artists'][0]['name'],
          'track_uri': track['uri']
      })

track_list_df = pd.DataFrame(track_list).drop_duplicates(subset=['track_uri']).drop_duplicates(subset=['track', 'artist'])

end_time = time.time()

print("Duration: " + str(end_time - start_time) + " seconds.")
print("Instances: " + str(track_list_df.shape[0]) + "\n")

track_list_df.head()

Duration: 21.1815025806427 seconds.
Instances: 1703



,track,artist,track_uri
0,Trains,Porcupine Tree,spotify:track:5lBQ3mnWsYIt5aCdgz1U9n
1,Thereafter,Abigail's Ghost,spotify:track:1y9D303WAMsztuPLBMfEvr
2,Sliver,Abigail's Ghost,spotify:track:4JSd7eQQCC1gKu3pmfkvGM
3,For Damien,Abigail's Ghost,spotify:track:0KtWZj6UkMTW4IsEA9YZg6
4,Bloodlust,Abigail's Ghost,spotify:track:2piyV7gACvmCYor6EwAT9j


In [0]:
start_time = time.time()

audio_analysis = []
track_list = list(track_list_df['track_uri'])

for index in range(0, len(track_list), 50):
  temp_audio_analysis = spotify.audio_features(track_list[index:index+50])
  audio_analysis += temp_audio_analysis

audio_analysis_df = pd.DataFrame(audio_analysis).dropna()
audio_analysis_df = audio_analysis_df.drop(['analysis_url', 'track_href', 'type', 'id', 'duration_ms', 'time_signature', 'liveness'], 
                                           axis=1).drop_duplicates(subset=['uri'])

end_time = time.time()

print("Duration: " + str(end_time - start_time) + " seconds.")
print("Instances: " + str(audio_analysis_df.shape[0]) + "\n")

audio_analysis_df.head()

retrying ...1secs
retrying ...1secs
Duration: 6.1008665561676025 seconds.
Instances: 1703



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,uri
0,0.456,0.776,9,-6.999,0,0.0348,0.042300,0.009750,0.223,174.261,spotify:track:5lBQ3mnWsYIt5aCdgz1U9n
1,0.154,0.871,10,-6.439,1,0.0897,0.081600,0.000150,0.354,172.138,spotify:track:1y9D303WAMsztuPLBMfEvr
2,0.406,0.789,5,-7.045,1,0.0438,0.000288,0.000263,0.410,159.995,spotify:track:4JSd7eQQCC1gKu3pmfkvGM
3,0.499,0.668,3,-7.795,1,0.0302,0.164000,0.000000,0.568,150.009,spotify:track:0KtWZj6UkMTW4IsEA9YZg6
4,0.590,0.722,7,-8.492,1,0.0386,0.010400,0.000082,0.512,120.007,spotify:track:2piyV7gACvmCYor6EwAT9j


In [0]:
dataset_df = pd.merge(track_list_df, audio_analysis_df, how='left', left_on='track_uri', right_on='uri').drop('uri', axis=1)
print("Instances: " + str(dataset_df.shape[0]) + "\n")
dataset_df.head()

Instances: 1703



,track,artist,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo
0,Trains,Porcupine Tree,spotify:track:5lBQ3mnWsYIt5aCdgz1U9n,0.456,0.776,9,-6.999,0,0.0348,0.042300,0.009750,0.223,174.261
1,Thereafter,Abigail's Ghost,spotify:track:1y9D303WAMsztuPLBMfEvr,0.154,0.871,10,-6.439,1,0.0897,0.081600,0.000150,0.354,172.138
2,Sliver,Abigail's Ghost,spotify:track:4JSd7eQQCC1gKu3pmfkvGM,0.406,0.789,5,-7.045,1,0.0438,0.000288,0.000263,0.410,159.995
3,For Damien,Abigail's Ghost,spotify:track:0KtWZj6UkMTW4IsEA9YZg6,0.499,0.668,3,-7.795,1,0.0302,0.164000,0.000000,0.568,150.009
4,Bloodlust,Abigail's Ghost,spotify:track:2piyV7gACvmCYor6EwAT9j,0.590,0.722,7,-8.492,1,0.0386,0.010400,0.000082,0.512,120.007


In [0]:
scaler = StandardScaler()

dataset_scaled = scaler.fit_transform(dataset_df.drop(['track', 'artist', 'track_uri'], axis = 1))
dataset_scaled_df = pd.DataFrame(dataset_scaled, columns = dataset_df.drop(['track', 'artist', 'track_uri'], axis = 1).columns)
dataset_scaled_df['track'] = dataset_df['track']
dataset_scaled_df['artist'] = dataset_df['artist']
dataset_scaled_df['track_uri'] = dataset_df['track_uri']

print("Instances: " + str(dataset_scaled_df.shape[0]) + "\n")
dataset_scaled_df.head()

Instances: 1703



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,track,artist,track_uri
0,0.222268,0.704402,1.073469,0.571318,-1.071233,-0.307258,-0.527370,-0.725475,-0.389246,1.886993,Trains,Porcupine Tree,spotify:track:5lBQ3mnWsYIt5aCdgz1U9n
1,-1.788162,1.122412,1.364154,0.711054,0.933503,0.502404,-0.391897,-0.754340,0.246301,1.810481,Thereafter,Abigail's Ghost,spotify:track:1y9D303WAMsztuPLBMfEvr
2,-0.110584,0.761603,-0.089271,0.559840,0.933503,-0.174526,-0.672192,-0.754000,0.517985,1.372853,Sliver,Abigail's Ghost,spotify:track:4JSd7eQQCC1gKu3pmfkvGM
3,0.508521,0.229190,-0.670641,0.372694,0.933503,-0.375098,-0.107851,-0.754791,1.284523,1.012962,For Damien,Abigail's Ghost,spotify:track:0KtWZj6UkMTW4IsEA9YZg6
4,1.114313,0.466796,0.492099,0.198772,0.933503,-0.251216,-0.637335,-0.754543,1.012838,-0.068298,Bloodlust,Abigail's Ghost,spotify:track:2piyV7gACvmCYor6EwAT9j


In [0]:
start_time = time.time()

clustering = KMeans(n_clusters=int(dataset_scaled_df.shape[0]/20)).fit(dataset_scaled_df.drop(['track', 'artist', 'track_uri', 'track_uri'], axis=1))

dataset_scaled_df['cluster'] = pd.Series(clustering.labels_)

end_time = time.time()

print("\nDuration: " + str(end_time - start_time))

cluster_of_song = int(dataset_scaled_df[dataset_scaled_df['track_uri'] == song_uri]['cluster'])
recommend_df = dataset_scaled_df[dataset_scaled_df['cluster'] == cluster_of_song]

feature = "energy"
recommend_df = recommend_df.sort_values(by=feature)

display(recommend_df)

recommended_tracks = []
df_index = [index for index in recommend_df.index]

center_index = df_index.index(0)
lower_index = center_index - 1
higher_index = center_index + 1


while(len(recommended_tracks) < 10 and (lower_index > 0 or higher_index < len(df_index))):
  lower_flag = False
  higher_flag = False

  if(lower_index <= 0):
    higher_flag = True
  elif(higher_index >= len(df_index)):
    lower_flag = True
  else:
    lower_val = abs(recommend_df.loc[df_index[lower_index], feature] - recommend_df.loc[df_index[center_index], feature])
    higher_val = abs(recommend_df.loc[df_index[higher_index], feature] - recommend_df.loc[df_index[center_index], feature])
    if(lower_val < higher_val):
      lower_flag = True
    else:
      higher_flag = True
  
  if(lower_flag):
    recommended_tracks.append(recommend_df.loc[df_index[lower_index]])
    lower_index -= 1
  elif(higher_flag):
    recommended_tracks.append(recommend_df.loc[df_index[higher_index]])
    higher_index += 1

recommended_tracks_df = pd.DataFrame(recommended_tracks) 
recommended_tracks_df


Duration: 1.1353349685668945


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,track,artist,track_uri,cluster
801,-0.083956,-0.114018,1.073469,0.726525,-1.071233,-0.339703,-0.341568,-0.637227,0.678084,1.946819,Wish You Were,Galahad,spotify:track:6sIaUgssOTl8v3xCGdW2Ed,43
0,0.222268,0.704402,1.073469,0.571318,-1.071233,-0.307258,-0.527370,-0.725475,-0.389246,1.886993,Trains,Porcupine Tree,spotify:track:5lBQ3mnWsYIt5aCdgz1U9n,43
1528,-0.583235,0.735202,1.654839,0.776181,-1.071233,-0.254165,-0.563565,-0.750762,0.508282,2.091806,Trying to Kiss the Sun,RPWL,spotify:track:2Uowsjz3WfYmnnBp78GmKg,43
813,-1.228969,0.920007,1.073469,0.851289,-1.071233,-0.152404,-0.629751,-0.754528,0.420955,0.559512,Perfection Personified,Galahad,spotify:track:5d5FyaGFAYBXQfZnwRDX4P,43
1383,-1.874703,0.937607,1.073469,0.101207,-1.071233,0.027521,-0.605621,0.408819,1.240859,2.383366,Futilists Lament - 2004 Remaster,High Tide,spotify:track:60OAfaa5MHopLjHu826Ip0,43
126,-0.929402,0.946407,1.654839,-0.118129,-1.071233,0.297408,-0.546674,-0.753994,0.673233,1.280591,The Thousand Days,IQ,spotify:track:699julCd4RCR76vrJazyhe,43
1267,-1.082514,0.955208,1.654839,0.759463,-1.071233,0.669056,-0.669828,-0.754614,-0.709446,1.865261,"I Rule The Ruins - (Live At Wacken, 2018)",Doro,spotify:track:3nEgq21KdnF2LC34JDBNq4,43
8,-1.169056,0.972808,1.364154,0.448301,-1.071233,0.098311,-0.672907,0.078077,2.274229,1.156471,Protist,Abigail's Ghost,spotify:track:21ohBMQMX5FefXDlLKN0RV,43
658,-0.822889,0.981608,1.654839,0.810616,-1.071233,-0.223195,-0.493933,-0.427056,0.018280,1.355410,Home,Pain of Salvation,spotify:track:1TbHJVNCASEwNbEej8uz9C,43
785,-0.796261,0.986008,1.654839,1.390021,-1.071233,-0.150930,-0.522544,0.865843,2.468290,1.816572,Nimue - A Mused Moment,Galahad,spotify:track:5sHFCVyAN9EOBzzjWJ9jdJ,43


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,valence,tempo,track,artist,track_uri,cluster
1528,-0.583235,0.735202,1.654839,0.776181,-1.071233,-0.254165,-0.563565,-0.750762,0.508282,2.091806,Trying to Kiss the Sun,RPWL,spotify:track:2Uowsjz3WfYmnnBp78GmKg,43
813,-1.228969,0.920007,1.073469,0.851289,-1.071233,-0.152404,-0.629751,-0.754528,0.420955,0.559512,Perfection Personified,Galahad,spotify:track:5d5FyaGFAYBXQfZnwRDX4P,43
1383,-1.874703,0.937607,1.073469,0.101207,-1.071233,0.027521,-0.605621,0.408819,1.240859,2.383366,Futilists Lament - 2004 Remaster,High Tide,spotify:track:60OAfaa5MHopLjHu826Ip0,43
126,-0.929402,0.946407,1.654839,-0.118129,-1.071233,0.297408,-0.546674,-0.753994,0.673233,1.280591,The Thousand Days,IQ,spotify:track:699julCd4RCR76vrJazyhe,43
1267,-1.082514,0.955208,1.654839,0.759463,-1.071233,0.669056,-0.669828,-0.754614,-0.709446,1.865261,"I Rule The Ruins - (Live At Wacken, 2018)",Doro,spotify:track:3nEgq21KdnF2LC34JDBNq4,43
8,-1.169056,0.972808,1.364154,0.448301,-1.071233,0.098311,-0.672907,0.078077,2.274229,1.156471,Protist,Abigail's Ghost,spotify:track:21ohBMQMX5FefXDlLKN0RV,43
658,-0.822889,0.981608,1.654839,0.810616,-1.071233,-0.223195,-0.493933,-0.427056,0.018280,1.355410,Home,Pain of Salvation,spotify:track:1TbHJVNCASEwNbEej8uz9C,43
785,-0.796261,0.986008,1.654839,1.390021,-1.071233,-0.150930,-0.522544,0.865843,2.468290,1.816572,Nimue - A Mused Moment,Galahad,spotify:track:5sHFCVyAN9EOBzzjWJ9jdJ,43
854,0.129069,1.043210,0.201414,0.807123,-1.071233,-0.274812,-0.670631,0.270508,0.663530,0.941387,Over & Out,Paatos,spotify:track:3L0YKIuMa6dpa3bvISymJS,43
1101,0.109098,1.087211,1.073469,1.014232,-1.071233,-0.270388,-0.672768,-0.216584,0.348182,0.654872,Serein,Katatonia,spotify:track:3ThZfByczQOSRlqLR2FGir,43
